# Week 10 Discussion Review

Using the UCI Pittsburgh Bridges data set:

In [2]:
import pandas as pd
import numpy as np

# load the data set: the shape attribute shows we have 108 rows & 13 columns
pb_df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/bridges/bridges.data.version1", header = None)
pb_df.shape

(108, 13)

In [3]:
# add meaningful column names
pb_df.columns = ['Identif', 
                'River',
                'Location',
                'Erected', 
                'Purpose',
                'Length',
                'Lanes',
                'Clear-G',
                'T-OR-D',
                'Material',
                'Span',
                'REL-L',
                'Type']

In [48]:
# sanity check
pb_df.head(10)

,Identif,River,Location,Erected,Purpose,Length,Lanes,Clear-G,T-OR-D,Material,Span,REL-L,Type
0,E1,M,3,1818,HIGHWAY,?,2,N,THROUGH,WOOD,SHORT,S,WOOD
1,E2,A,25,1819,HIGHWAY,1037,2,N,THROUGH,WOOD,SHORT,S,WOOD
2,E3,A,39,1829,AQUEDUCT,?,1,N,THROUGH,WOOD,?,S,WOOD
3,E5,A,29,1837,HIGHWAY,1000,2,N,THROUGH,WOOD,SHORT,S,WOOD
4,E6,M,23,1838,HIGHWAY,?,2,N,THROUGH,WOOD,?,S,WOOD
5,E7,A,27,1840,HIGHWAY,990,2,N,THROUGH,WOOD,MEDIUM,S,WOOD
6,E8,A,28,1844,AQUEDUCT,1000,1,N,THROUGH,IRON,SHORT,S,SUSPEN
7,E9,M,3,1846,HIGHWAY,1500,2,N,THROUGH,IRON,SHORT,S,SUSPEN
8,E10,A,39,1848,AQUEDUCT,?,1,N,DECK,WOOD,?,S,WOOD
9,E11,A,29,1851,HIGHWAY,1000,2,N,THROUGH,WOOD,MEDIUM,S,WOOD


In [51]:
# how many '?' appear in Length column?
sum(pb_df['Length'] == '?')

27

__First Task__: You’ve been asked to generate a quick report that tells us how many bridges of each ‘Purpose’/’Material’ grouping within the data set have been constructed over each of the rivers listed in the data set. For each river, your output should include the Purpose, Material, and count (aka ‘How Many?’), similar to the output shown in the graphic below for River 'A', and your report should include similar content for each of the rivers contained within the data set.

In [4]:
# grouping by River, Purpose, Material and calc total number of items in each grouping
bridges1 = pd.DataFrame(pb_df.groupby([pb_df['River'], pb_df['Purpose'], pb_df['Material']]).size() )
bridges1

0
River Purpose  Material    
A     AQUEDUCT IRON       1
               WOOD       3
      HIGHWAY  ?          1
               IRON       2
               STEEL     21
               WOOD       8
      RR       IRON       1
               STEEL      9
               WOOD       2
      WALK     STEEL      1
M     HIGHWAY  IRON       4
               STEEL     19
               WOOD       3
      RR       IRON       2
               STEEL     13
O     HIGHWAY  ?          1
               IRON       1
               STEEL      9
      RR       STEEL      4
Y     HIGHWAY  STEEL      2
      RR       STEEL      1

In [53]:
bridges1.rename(columns={0: "How Many?"}, inplace = True)
bridges1

How Many?
River Purpose  Material           
A     AQUEDUCT IRON              1
               WOOD              3
      HIGHWAY  ?                 1
               IRON              2
               STEEL            21
               WOOD              8
      RR       IRON              1
               STEEL             9
               WOOD              2
      WALK     STEEL             1
M     HIGHWAY  IRON              4
               STEEL            19
               WOOD              3
      RR       IRON              2
               STEEL            13
O     HIGHWAY  ?                 1
               IRON              1
               STEEL             9
      RR       STEEL             4
Y     HIGHWAY  STEEL             2
      RR       STEEL             1

## Week 10 Discussion Item Part 2

You’ve been asked to generate a second report that shows the average length for each ‘Purpose’/’Material’ bridge grouping within the data set. As you should recall from our previous work with the Pittsburgh Bridges data set, the ‘Length’ attribute is not provided to us in a numeric format and also contains many missing values. As such, you should clean up the contents of that column and convert it to numeric format before attempting to generate your report.

In [9]:
# For the Length variable we need to replace the '?' with NaN before doing any computations:
pb_df["Length"].replace({'?':np.nan}, inplace = True)

In [10]:
# convert the data type of the Length column to numeric
pb_df["Length"] = pd.to_numeric(pb_df["Length"])

In [56]:
# group by Purpose and Material
grouped = pb_df.groupby(['Purpose', 'Material'])

# now lets work with just the Length component of the groupby object
# we've created: 
# compute the mean Length for each Purpose/Material combination
grouped_length = grouped['Length']
# grouped_length.agg({'Average Length' : 'mean'})
bridges2 = pd.DataFrame(grouped_length.agg(['mean'])) 
bridges2.rename(columns={'mean': 'Average Length'}, inplace = True)
bridges2

Average Length
Purpose  Material                
AQUEDUCT IRON         1000.000000
         WOOD         1092.000000
HIGHWAY  ?                    NaN
         IRON         1216.666667
         STEEL        1557.804348
         WOOD         1053.375000
RR       IRON         1100.000000
         STEEL        1946.850000
         WOOD                 NaN
WALK     STEEL                NaN

## Week 10 Discussion Part 3

Finally, you’ve been asked to generate one last report that shows the average length, count, minimum length, and maximum length of bridges built during 4 equal length time periods (1818 – 1860; 1860-1902; 1902-1944; 1944-1986). 

In [60]:
# split the Erected column into 4 separate bins of equal numeric range
quartiles = pd.cut(pb_df.Erected, 4, precision = 0)
quartiles[:5]

0    (1818.0, 1860.0]
1    (1818.0, 1860.0]
2    (1818.0, 1860.0]
3    (1818.0, 1860.0]
4    (1818.0, 1860.0]
Name: Erected, dtype: category
Categories (4, interval[float64]): [(1818.0, 1860.0] < (1860.0, 1902.0] < (1902.0, 1944.0] < (1944.0, 1986.0]]

In [42]:
# define a function that specifies the aggregating functions we want to apply
# to the 'Length' values contained within each of our Erected quartile cuts
# Note the use of a dict object
def get_stats(group):
    return {'Min Length': group.min(), 'Max Length': group.max(),
            'Count': group.count(), 'Average Length': group.mean()}


In [61]:
# group the data2 column's values relative to the data1 quartile cuts we made
# above
grouped = pb_df.Length.groupby(quartiles)

# apply the get_stats function to the grouped data2 values
# In the results we see the data1 quartile cuts are used as the row index labels
grouped.apply(get_stats).unstack()

,Average Length,Count,Max Length,Min Length
Erected,,,,
"(1818.0, 1860.0]",1094.625000,8.0,1500.0,990.0
"(1860.0, 1902.0]",1603.347826,23.0,4558.0,1000.0
"(1902.0, 1944.0]",1676.181818,33.0,3000.0,860.0
"(1944.0, 1986.0]",1530.411765,17.0,3756.0,804.0
